In [1]:
import os
import shutil

### preparing data into directories

Directories for the training, validation and test splits

In [2]:
full_data_dir = '../data/CatVsDogs/train'
base_dir = '../data/cat&dogs'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

os.mkdir(base_dir)
os.mkdir(train_dir)
os.mkdir(val_dir)
os.mkdir(test_dir)

Directory with training cat pictures

In [3]:
train_cat_dir = os.path.join(train_dir, 'cat')
os.mkdir(train_cat_dir)

Directory with training dog pictures

In [4]:
train_dog_dir = os.path.join(train_dir, 'dog')
os.mkdir(train_dog_dir)

Directory with validation cat pictures

In [5]:
val_cat_dir = os.path.join(val_dir, 'cat')
os.mkdir(val_cat_dir)

Directory with validation dog pictures

In [6]:
val_dog_dir = os.path.join(val_dir, 'dog')
os.mkdir(val_dog_dir)

Directory with testing cat pictures

In [7]:
test_cat_dir = os.path.join(test_dir, 'cat')
os.mkdir(test_cat_dir)

Directory with testing dog pictures

In [8]:
test_dog_dir = os.path.join(test_dir, 'dog')
os.mkdir(test_dog_dir)

### Putting images to folders

In [9]:
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]

for fname in fnames:
    src = os.path.join(full_data_dir, fname)
    dst = os.path.join(train_cat_dir, fname)
    shutil.copyfile(src, dst)

In [10]:
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]

for fname in fnames:
    src = os.path.join(full_data_dir, fname)
    dst = os.path.join(val_cat_dir, fname)
    shutil.copyfile(src, dst)

In [11]:
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]

for fname in fnames:
    src = os.path.join(full_data_dir, fname)
    dst = os.path.join(test_cat_dir, fname)
    shutil.copyfile(src, dst)

In [12]:
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]

for fname in fnames:
    src = os.path.join(full_data_dir, fname)
    dst = os.path.join(train_dog_dir, fname)
    shutil.copyfile(src, dst)

In [13]:
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]

for fname in fnames:
    src = os.path.join(full_data_dir, fname)
    dst = os.path.join(val_dog_dir, fname)
    shutil.copyfile(src, dst)

In [14]:
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]

for fname in fnames:
    src = os.path.join(full_data_dir, fname)
    dst = os.path.join(test_dog_dir, fname)
    shutil.copyfile(src, dst)